In [1]:
!pip install paho-mqtt
!pip install wfdb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.2/67.2 kB 639.4 kB/s eta 0:00:00


In [8]:
import wfdb
import numpy as np
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import paho.mqtt.client as mqtt
import time
from collections import deque

In [18]:
if os.path.isdir("mitdb"):
    print('You already have the data.')
else:
    wfdb.dl_database('mitdb', 'mitdb')


You already have the data.


In [44]:
def load_data(record_name):
    try:
        print(f"Loading record: {record_name}")
        record = wfdb.rdrecord(record_name, sampto=300000)
        annotation = wfdb.rdann(record_name, 'atr', sampto=300000)

        # 获取心电信号和注释
        ecg_signal = record.p_signal[:, 0]
        annotations = annotation.sample

        print(f"ECG signal length: {len(ecg_signal)}")
        print(f"Number of annotations: {len(annotations)}")

        # 将心电信号分成每分钟的段落，并计算每分钟心跳数
        fs = record.fs  # 采样频率
        samples_per_minute = int(fs * 60)
        num_minutes = len(ecg_signal) // samples_per_minute

        heartbeat_counts = []

        for i in range(num_minutes):
            start = i * samples_per_minute
            end = (i + 1) * samples_per_minute
            segment_annotations = annotations[(annotations >= start) & (annotations < end)]
            heartbeat_counts.append(len(segment_annotations))

        print(f"Heartbeat counts: {heartbeat_counts}")

        return np.array(heartbeat_counts).reshape(-1, 1)
    except Exception as e:
        print(f"Error loading data: {e}")
        return np.array([]).reshape(-1, 1)


In [52]:
# 加载数据
data = load_data("mitdb/100")

# 检查数据是否为空
if data.size == 0:
    print("No data loaded. Please check the record name and path.")
else:


    # 假设正常心跳在60到100次之间
    labels = np.where((data >=70) & (data <= 75), 0, 1)  # 0: 正常, 1: 异常

    # 检查样本数量是否足够进行训练和测试集划分
    if len(data) < 10:  # 这里假设至少需要10个样本
        print("Not enough data to split into training and testing sets.")
    else:
        # 划分数据集 60%训练, 40%测试
        X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.4, random_state=42)

        model = train_svm(X_train, y_train)

        # 评估模型
        y_pred = model.predict(X_test)
        print(f"Accuracy: {accuracy_score(y_test, y_pred)}")
        print(classification_report(y_test, y_pred))


Accuracy: 0.6666666666666666
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.67      1.00      0.80         4

    accuracy                           0.67         6
   macro avg       0.33      0.50      0.40         6
weighted avg       0.44      0.67      0.53         6



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precisio

In [53]:
broker1_address = "broker.emqx.io"
broker2_address = "test.mosquitto.org"
port = 1883
topic = "patient/+/heartbeat"  # 使用通配符来订阅多个主题

# 定义桥接模式: "in", "out", "both"
bridge_mode = "in"

class BridgeMQTTClient(mqtt.Client):
    def __init__(self, cname, **kwargs):
        super(BridgeMQTTClient, self).__init__(**kwargs)
        self.cname = cname
        self.on_connect = self.on_connect_callback
        self.on_message = self.on_message_callback
        self.processed_messages = deque(maxlen=100)

    def on_connect_callback(self, client, userdata, flags, rc):
        if rc == 0:
            print(f"{self.cname} connected to broker")
            if (self.cname == "bridge_c1" and bridge_mode in ["in", "both"]) or \
               (self.cname == "bridge_c2" and bridge_mode in ["out", "both"]):
                self.subscribe(topic)
        else:
            print(f"Connection failed with code {rc} for {self.cname}")

    def on_message_callback(self, client, userdata, msg):
        m_decode = str(msg.payload.decode("utf-8"))

        # 处理并发布消息
        print(f"Received message from {self.cname}: {m_decode}")
        heartbeat = int(m_decode)
        prediction = model.predict([[heartbeat]])
        if prediction == 0:
            print(f"Heartbeat from {msg.topic} is normal")
        else:
            print(f"Heartbeat from {msg.topic} is abnormal")

        if self.cname == "bridge_c1" and bridge_mode in ["in", "both"]:
            bridge_client2.publish(msg.topic, m_decode)  # 保持原主题
            print(f"Forwarded message to broker2: {m_decode}")

        elif self.cname == "bridge_c2" and bridge_mode in ["out", "both"]:
            bridge_client1.publish(msg.topic, m_decode)  # 保持原主题
            print(f"Forwarded message to broker1: {m_decode}")

# 初始化第一个客户端，订阅并转发消息
bridge_client1 = BridgeMQTTClient(cname="bridge_c1")
bridge_client1.connect(broker1_address, port)
bridge_client1.loop_start()

# 初始化第二个客户端，接收转发的消息
bridge_client2 = BridgeMQTTClient(cname="bridge_c2")
bridge_client2.connect(broker2_address, port)
bridge_client2.loop_start()

# 保持主线程运行
try:
    while True:
        time.sleep(4)
except KeyboardInterrupt:
    pass
finally:
    print('now stop')
    bridge_client1.loop_stop()
    bridge_client2.loop_stop()
    bridge_client1.disconnect()
    bridge_client2.disconnect()

<ipython-input-53-4c78830a4350>:11: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  super(BridgeMQTTClient, self).__init__(**kwargs)


bridge_c1 connected to broker
bridge_c2 connected to broker
Received message from bridge_c1: 78
Heartbeat from patient/num1/heartbeat is abnormal
Forwarded message to broker2: 78
Received message from bridge_c1: 76
Heartbeat from patient/num3/heartbeat is abnormal
Forwarded message to broker2: 76
Received message from bridge_c1: 83
Heartbeat from patient/num2/heartbeat is abnormal
Forwarded message to broker2: 83
Received message from bridge_c1: 67
Heartbeat from patient/num1/heartbeat is normal
Forwarded message to broker2: 67
Received message from bridge_c1: 84
Heartbeat from patient/num3/heartbeat is abnormal
Forwarded message to broker2: 84
Received message from bridge_c1: 98
Heartbeat from patient/num2/heartbeat is abnormal
Forwarded message to broker2: 98
Received message from bridge_c1: 129
Heartbeat from patient/num1/heartbeat is abnormal
Forwarded message to broker2: 129
Received message from bridge_c1: 94
Heartbeat from patient/num3/heartbeat is abnormal
Forwarded message to 